# Exercise set 9

> *  The goal of the first part of the exercise is to gain familiarity with partial least
squares regression. For this, we will make
a model that can predict the concentrations in a mixture from near-infrared spectra.
> 
> * In the second part of the exercise, you will do a PCA analysis of gene expressions.
This part is to get more experience with PCA, particularly interpreting results from PCA.

## Exercise 9.1

[Windig and Stephenson](https://doi.org/10.1021/ac00046a015) measured near-infrared spectra
for 140 mixtures of the solvents methylene chloride, 2-butanol, methanol,
dichloropropane, and acetone.
We will here predict the compositions of the mixtures from the spectra.
Each spectrum was sampled at 700 wavelengths
between 1100 and 2500~nm. The file
[`Data/windig.csv`](Data/windig.csv) contains the raw data:
Each row in this file
contains a spectrum (the columns starting with `wavelength.`) and the
corresponding concentrations (the columns starting with `conc.`).

The data can be loaded as follows:

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context("notebook")

data = pd.read_csv("Data/windig.csv")
X = data.filter(like="wavelength", axis=1).values  # NIR spectra
Y = data.filter(like="conc", axis=1).values  # Concentrations
print(f"No. of spectra: {X.shape[0]}")
print(f"No. of wavelengths: {X.shape[1]}")
print(f"No of concentration samples: {Y.shape[0]}")
print(f"No of species in each sample: {Y.shape[1]}")

In [ ]:
# To plot the spectra:
fig, ax = plt.subplots(constrained_layout=True)
for spectrum in X:
    ax.plot(spectrum)
ax.set(xlabel="Wavelength (no unit)", ylabel="Absorbance")
sns.despine(fig=fig)

### 9.1(a)
Create a partial least squares regression (PLSR) model for predicting
the concentrations. Use 1 PLS component for your first model and
assess it using $R^2$, RMSEC, RMSECV and RMSEP.  An example
of how this can be done are given below.

These values (RMSEC, RMSECV, and RMSEP) are all based on calculating the
root mean squared error (RMSE) given by,

\begin{equation}
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^N (y_i - \hat{y}_i)^2},
\tag{1}\end{equation}

where $y_i$ are our measured $y$-values and $\hat{y}_i$ are the
values predicted by our model. The difference between RMSEC,
RMSEP, and RMSECV lie in the part of the data we use to
calculate them. This is based on first splitting the data into
a *training* and *test* set, and then
performing what we call
[*cross-validation*](https://scikit-learn.org/stable/modules/cross_validation.html) using
the training set:


* When we use the training set to create our model, we are doing
  a *calibration*. If we calculate RMSE based on using
  the training set, we refer to this as the RMSEC (root mean squared
  error of calibration).
  This number
  quantifies the error we get in connection with making (calibrating)
  the model.
  
* When we use the test set to test our model, we are
  checking how well our model *predicts* "new" samples
  (that is, samples not used when making the model). If
  we calculate RMSE based on the training set, we refer to 
  this as the RMSEP (root mean squared error of prediction). This
  number quantifies the error we can expect to make when using
  our model for predicting new samples.
  
* Cross-validation is based on further splitting the training set. Typically, we divide the     training set into $k$ smaller subsamples, and we repeat the fitting of the
  model $k$ times. Each time we repeat the fitting, we retain a single
  subsample for validation, and we fit the model using
  the $k-1$ other subsamples. For the subsample we retained
  for validation, we can calculate the RMSE value of how
  well our model predicts it. Since we repeat this $k$ times,
  we can make it so that each of the $k$ subsamples is
  used exactly once for validation. Finally, we can
  obtain the average RMSE of the $k$ fittings, and we
  refer to this value as the RMSECV (root mean squared error
  of cross-validation). This number also estimates how well the model predicts new cases, and   we also get information on how sensitive the model is to model parameters and the part of   
  the training set used. We can also use cross-validation to optimize the parameters in the 
  model (for instance, the number of PLS components).

Luckily, methods for splitting our data into training and test
sets, calculating RMSE, and doing cross-validation are already
available in sklearn. Note: Splitting the data into training
and test sets and performing cross-validation involves some randomness, and your answers will probably change if you rerun
your code.

Here is a visualization of the splitting:
![cross](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [ ]:
# First, here is how you can split into a testing and training set:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.33,  # Use 33 % of the data (one third) for the test set.
    shuffle=True,  # Randomly shuffle the data
)

In [ ]:
# Make a PLS model:
from sklearn.cross_decomposition import PLSRegression

model = PLSRegression(n_components=4)  # Set up a PLS model with 4 components
model.fit(X_train, Y_train)  # Fit/make the model

In [ ]:
# Calulate scores for the model:
from sklearn.metrics import r2_score, mean_squared_error

Y_hat_train = model.predict(X_train)  # Predict for the training set
Y_hat_test = model.predict(X_test)  # Predict for the test set

# Calculate R²:
r2_train = r2_score(Y_train, Y_hat_train)
print(f"R² for training set: {r2_train}")
r2_test = r2_score(Y_test, Y_hat_test)
print(f"R² for test set: {r2_test}")

# Calculate RMSE:
rmsec = mean_squared_error(Y_train, Y_hat_train, squared=False)
print(f"RMSEC: {rmsec}")
rmsep = mean_squared_error(Y_test, Y_hat_test, squared=False)
print(f"RMSEP: {rmsep}")

In [ ]:
# Example for cross-validation:
from sklearn.model_selection import cross_val_score

# Run cross-validation:
cvscore = cross_val_score(
    model,  # Select the model we are going to score
    X_train,  # Give the X-training set
    Y_train,  # Give the y-training set
    scoring="neg_mean_squared_error",  # select scoring method
    cv=5,  # Number of splits to make
)
# Note: the scoring is here "neg_mean_squared_error".
# This is the negative of the MSE!
# The cross_val_score method is often used in
# connection with optimization where we would like to
# maximize something, and the score can be used to pick
# the best value. Since we usually do not want to
# maximize the error, this method is made so that it
# calculates the negative of the error.

cvscore = np.sqrt(-cvscore)  # Account for the negative sign.
rmsecv = cvscore.mean()
rmsecv_std = np.std(cvscore)
print(f"\nRMSECV: {rmsecv} ± {rmsecv_std}")

In [ ]:
# Your code here

#### Your answer to question 9.1(a): ($R^2$, RMSEC, RMSECV and RMSEP)
*Double click here*

### 9.1(b)
Improve your PLSR model by including more
PLS components.
Try components from 2 up to 15 and compare the different models. How many
PLS components are you satisfied with? 

In [ ]:
# Your code here

#### Your answer to question 9.1(b):
*Double click here*

### 9.1(c)
Plot the regression coefficients for the model you found in [9.1(b)](#9.1(b)).

In [ ]:
# You can get the regression coefficients with:
B = model.coef_
# To get the coefficients for solvent no. i, you can do:
# B[:, i]  # this selects all rows for column i.

In [ ]:
# Your code here

#### Your answer to question 9.1(c):
*Double click here*

### 9.1(d)
Optimize the number of PLS components by a cross-validated grid search of the number of
PLS components. Is this optimized model different from the PLS model you found in [9.1(b)](#9.1(b))?

In [ ]:
# Here is the code for running the optimalization. This will try out
# the number of PLS components and score the model with cross validation.
from sklearn.model_selection import GridSearchCV

# First, we define a range of PLS components to try, let us
# do 1, 2, ..., 20, 25, 50, 75, 100:
parameters = {
    "n_components": list(range(1, 20)) + [25, 50, 75, 100]
}

grid = GridSearchCV(
    PLSRegression(),  # the model we will make
    parameters,  # the parameters to investigate
    cv=5,  # number of splits for cross-validation
    scoring="r2",  # select the model with highest R²
    refit=True,  # refit the best model for the whole training set
)
grid.fit(X_train, Y_train)

In [ ]:
# The best estimator us:
grid.best_estimator_

In [ ]:
# It is also a good idea to plot the scores to see where it levels off:
fig, ax = plt.subplots(constrained_layout=True)
ax.errorbar(
    parameters["n_components"],
    grid.cv_results_["mean_test_score"],
    yerr=grid.cv_results_["std_test_score"],
    marker="o",
)
ax.set(xlabel="No. of PLS components", ylabel="Test score (R²)")
sns.despine(fig=fig)
# Hint: It may be a good idea to zoom in on the part (1, 20) for the x-axis:

#### Your answer to question 9.1(d):
*Double click here*

### 9.1(e)
Assume that you are given a spectrum from a mixture with unknown concentrations of the solvents. How well would your model
predict the unknown concentrations?

In [ ]:
# Your code here

#### Your answer to question 9.1(e):
*Double click here*

### 9.1(f)
Create a least squares model for predicting the concentrations.
Assess it using $R^2$, RMSEC, RMSECV and RMSEP. Does this model
perform as you expect?

In [ ]:
# Your code here

#### Your answer to question 9.1(f):
*Double click here*

## Exercise 9.2

[Schummer *et al.*](https://doi.org/10.1016/S0378-1119(99)00342-X) studied ovarian cancer by measuring gene expression values for 1536 genes in both normal and cancer tissues. One of their goals was to investigate if specific genes were overexpressed in cancer samples compared with normal samples.
This knowledge may be used for diagnosis, and we will here see if we
can find such genes by performing a PCA. The raw data can be
found in the file [`Data/ovo.csv`](Data/ovo.csv).
Each row in the data file contains a tissue sample's gene expressions (for 1536 genes). Each column corresponds to a specific gene, named `X.1`, `X.2`, and so on.
The classification of tissue as normal (`N`) or cancer (`C`) can
be found in the column `class`. The data can be loaded with:

In [ ]:
"""Load the data set."""
import pandas as pd

data = pd.read_csv("Data/ovo.csv")
classes = data["class"]  # Classification of samples.
X = data.filter(like="X.", axis=1)  # Gene expressions for samples.

### 9.2(a)
 
Center the data before performing the PCA. This can be
done as follows with the `scale` method
from `sklearn.preprocessing`: `X = scale(X, with_std=False)`.
Here, all the variables are in the same units, so we do not need
to scale the variance (we set `with_std=False`).

In [ ]:
from sklearn.preprocessing import scale
X = scale(X, with_std=False)

In [ ]:
# Your code here

#### Your answer to question 9.2(a):
*Double click here*

### 9.2(b)
Inspect the data by plotting the scores and loadings for
principal component
number 1 and principal component number 2:


* (i) Can you observe any clustering
  of the samples? Here, it may be helpful to colour the samples
  according to their classification as normal or cancer.


* (ii) Are there any outliers among the samples?


* (iii) Can you identify some
  genes which are overexpressed in tumors? 


* (iv) Can you identify some
  genes which are underexpressed in tumors? 

In [ ]:
# Your code here

#### Your answer to question 9.2(b):
*Double click here*

### 9.2(c)
Based on your answer in [9.2(b)](#9.2(b)), can
you identify some pairs of genes that seem to distinguish between
normal and tumor tissues? Support your findings by plotting the raw data.

In [ ]:
# Your code here

#### Your answer to question 9.2(c):
*Double click here*